In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
mpsolution = mp.solutions.holistic
drawing = mp.solutions.drawing_utils

In [3]:
def Superimpose(image, landmarks):
    drawing.draw_landmarks(image, landmarks.face_landmarks, mpsolution.FACEMESH_CONTOURS, drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1), drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    drawing.draw_landmarks(image, landmarks.pose_landmarks, mpsolution.POSE_CONNECTIONS)
    drawing.draw_landmarks(image, landmarks.left_hand_landmarks, mpsolution.HAND_CONNECTIONS, drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1), drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    drawing.draw_landmarks(image, landmarks.right_hand_landmarks, mpsolution.HAND_CONNECTIONS, drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1), drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))

In [4]:
def BrightnessContrast(img, brightness = 0, contrast = 0):
     
    # getTrackbarPos returns the current
    # position of the specified trackbar.
    # brightness = cv2.getTrackbarPos('Brightness', 'test')
      
    # contrast = cv2.getTrackbarPos('Contrast', 'test')
  
    effect = controller(img, brightness, contrast)
  
    # The function imshow displays an image
    # in the specified window
    return effect

def controller(img, brightness=255, contrast=127):
    brightness = int((brightness - 0) * (255 - (-255)) / (510 - 0) + (-255))
    contrast = int((contrast - 0) * (127 - (-127)) / (254 - 0) + (-127))
  
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            max = 255
        else:
            shadow = 0
            max = 255 + brightness

        al_pha = (max - shadow) / 255
        ga_mma = shadow
  
        # The function addWeighted calculates
        # the weighted sum of two arrays
        cal = cv2.addWeighted(img, al_pha, img, 0, ga_mma)
    else:
        cal = img
  
    if contrast != 0:
        Alpha = float(131 * (contrast + 127)) / (127 * (131 - contrast))
        Gamma = 127 * (1 - Alpha)  
        # The function addWeighted calculates
        # the weighted sum of two arrays
        cal = cv2.addWeighted(cal, Alpha, cal, 0, Gamma)
  
    # putText renders the specified text string in the image.
    #cv2.putText(cal, 'B:{},C:{}'.format(brightness, contrast), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  
    return cal

In [5]:
def detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    result = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return result

In [74]:
def Process(result):
    processed = []
    temp = list(result.pose_landmarks.landmark)[0]
    processed = np.array([[(res.x - temp.x), (res.y - temp.y), (res.z - temp.z), res.visibility] for res in result.pose_landmarks.landmark])
    
    return processed.flatten()

In [7]:
video = cv2.VideoCapture(0)
# cv2.namedWindow('test')
# cv2.createTrackbar('Brightness', 'test', 255, 2 * 255, BrightnessContrast) 
# cv2.createTrackbar('Contrast', 'test', 127, 2 * 127, BrightnessContrast)

with mpsolution.Holistic(min_detection_confidence = 0.5, min_tracking_confidence=0.5) as model:
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        result1 = detection(frame, model)
        effect = BrightnessContrast(frame, 357, 209)
        result2 = detection(effect, model)

        Superimpose(frame, result1)
        Superimpose(effect, result2)
        
        cv2.imshow("OnlyFans!!", frame)
        cv2.imshow("effect", effect)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    video.release()
    cv2.destroyAllWindows()

In [20]:
result2.pose_landmarks.landmark

[x: 0.40962904691696167
y: 0.6593536734580994
z: -0.7133809328079224
visibility: 0.9999851584434509
, x: 0.4415042996406555
y: 0.6145055890083313
z: -0.6552112102508545
visibility: 0.9999768733978271
, x: 0.45622459053993225
y: 0.6207953095436096
z: -0.6549965143203735
visibility: 0.9999798536300659
, x: 0.4684542715549469
y: 0.6267510056495667
z: -0.6546993255615234
visibility: 0.9999721050262451
, x: 0.3955652117729187
y: 0.5939257144927979
z: -0.6980414986610413
visibility: 0.9999784231185913
, x: 0.38107234239578247
y: 0.5877246260643005
z: -0.697708785533905
visibility: 0.9999784827232361
, x: 0.3665107786655426
y: 0.5827690362930298
z: -0.6979380249977112
visibility: 0.9999675750732422
, x: 0.46887022256851196
y: 0.6498232483863831
z: -0.3204565644264221
visibility: 0.9999771118164062
, x: 0.3299427628517151
y: 0.5979783535003662
z: -0.5186215043067932
visibility: 0.999992847442627
, x: 0.42008939385414124
y: 0.7129100561141968
z: -0.5881872177124023
visibility: 0.999975562095642

In [76]:
test = Process(result2)
test

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  9.99985158e-01,
        3.18752527e-02, -4.48480844e-02,  5.81697226e-02,  9.99976873e-01,
        4.65955436e-02, -3.85583639e-02,  5.83844185e-02,  9.99979854e-01,
        5.88252246e-02, -3.26026678e-02,  5.86816072e-02,  9.99972105e-01,
       -1.40638351e-02, -6.54279590e-02,  1.53394341e-02,  9.99978423e-01,
       -2.85567045e-02, -7.16290474e-02,  1.56721473e-02,  9.99978483e-01,
       -4.31182683e-02, -7.65846372e-02,  1.54429078e-02,  9.99967575e-01,
        5.92411757e-02, -9.53042507e-03,  3.92924368e-01,  9.99977112e-01,
       -7.96862841e-02, -6.13753200e-02,  1.94759429e-01,  9.99992847e-01,
        1.04603469e-02,  5.35563827e-02,  1.25193715e-01,  9.99975562e-01,
       -3.64268124e-02,  4.18767929e-02,  6.64389729e-02,  9.99990404e-01,
        7.72701502e-02,  2.19542503e-01,  5.91750778e-01,  9.99741673e-01,
       -2.50905156e-01,  1.85146272e-01,  3.91603172e-01,  9.97557878e-01,
        3.04415703e-01,  